In [7]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2
train_path='train'
test_path='test'

class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=2)
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
      
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool(output)
        output=self.conv2(output)
        output=self.relu2(output)
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output) 
        output=output.view(-1,32*75*75)  
        output=self.fc(output)
        return output
    
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
checkpoint=torch.load('checkpoint.model')
model=ConvNet(num_classes=6)
model.load_state_dict(checkpoint)
model.eval()
transformer=transforms.Compose([
    transforms.Resize([150,150]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                       [0.5,0.5,0.5])
])


def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

images_path=glob.glob(test_path+'/**/*.jpeg')
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)
pred_dict

{'test\\test\\a.jpeg': 'X-Ray',
 'test\\test\\b.jpeg': 'X-Ray',
 'test\\test\\download.jpeg': 'X-Ray',
 'test\\test\\image16.jpeg': 'ECG',
 'test\\test\\image2.jpeg': 'Report',
 'test\\test\\image21.jpeg': 'X-Ray',
 'test\\test\\image29.jpeg': 'ECG',
 'test\\test\\image3.jpeg': 'Presciption',
 'test\\test\\image30.jpeg': 'ECG',
 'test\\test\\image35.jpeg': 'ECG',
 'test\\test\\image39.jpeg': 'ECG',
 'test\\test\\image47.jpeg': 'X-Ray',
 'test\\test\\image59.jpeg': 'ECG',
 'test\\test\\image8.jpeg': 'Report',
 'test\\test\\image85.jpeg': 'ECG',
 'test\\test\\image87.jpeg': 'X-Ray',
 'test\\test\\image90.jpeg': 'X-Ray',
 'test\\test\\re1.jpeg': 'Report',
 'test\\test\\re2.jpeg': 'ECG'}